In [1]:
import torch, torchvision, PIL, numpy as np
import matplotlib.pyplot as plt
import pathlib
import PIL
import matplotlib.patches as patches
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#test to make sure i have the right directory
plt.imshow(PIL.Image.open('/content/drive/MyDrive/CPEN291/Final/freiburg_groceries_dataset/images/BEANS/BEANS0000.png'))

In [3]:
ds_dir = '/content/drive/MyDrive/CPEN291/Final/freiburg_groceries_dataset'

In [59]:
xform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
ds = torchvision.datasets.ImageFolder(ds_dir, transform=xform)

In [60]:
n_all = len(ds)
n_train = int(0.8 * n_all)
n_test = n_all - n_train

rng = torch.Generator().manual_seed(291)
ds_train, ds_test = torch.utils.data.random_split(ds, [n_train, n_test], rng)

In [61]:
loader_train = torch.utils.data.DataLoader(ds_train, batch_size = 4, shuffle=True)
loader_test = torch.utils.data.DataLoader(ds_test, batch_size = 4, shuffle=True)

In [62]:
model = torchvision.models.inception_v3(pretrained=True)
model.fc
model.fc = torch.nn.Linear(model.fc.in_features, 25)
torch.nn.init.xavier_uniform_(model.fc.weight)

Parameter containing:
tensor([[-0.0413,  0.0382,  0.0166,  ...,  0.0199,  0.0281,  0.0313],
        [ 0.0115, -0.0171, -0.0459,  ..., -0.0088,  0.0041, -0.0198],
        [ 0.0502,  0.0012,  0.0263,  ..., -0.0382,  0.0152, -0.0275],
        ...,
        [-0.0264,  0.0382,  0.0406,  ...,  0.0338,  0.0172,  0.0086],
        [ 0.0268,  0.0195,  0.0381,  ...,  0.0021,  0.0073,  0.0154],
        [-0.0288,  0.0492, -0.0202,  ...,  0.0319,  0.0349,  0.0312]],
       requires_grad=True)

In [63]:
device = torch.device('cuda:0')
model = model.to(device)

In [64]:
criterion = torch.nn.CrossEntropyLoss()

def run_test(model):
    nsamples_test = len(ds_test)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for samples, labels in loader_test:
            samples = samples.to(device)
            labels = labels.to(device)
            outs = model(samples)
            loss += criterion(outs, labels)
            _, preds = torch.max(outs.detach(), 1)
            correct_mask = preds == labels
            correct += correct_mask.sum(0).item()
    return loss / nsamples_test, correct / nsamples_test

In [65]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [66]:
def run_train(model, opt, sched):
    nsamples_train = len(ds_train)
    loss_sofar, correct_sofar = 0, 0
    model.train()
    with torch.enable_grad():
        for samples, labels in loader_train:
            samples = samples.to(device)
            labels = labels.to(device)
            opt.zero_grad()
            outs = model(samples)
            _, preds = torch.max(outs.detach(), 1)
            loss = criterion(outs, labels)
            loss.backward()
            opt.step()
            loss_sofar += loss.item() * samples.size(0)
            correct_sofar += torch.sum(preds == labels.detach())
    sched.step()
    return loss_sofar / nsamples_train, correct_sofar / nsamples_train

In [67]:
def run_all(model, optimizer, scheduler, n_epochs):
    for epoch in range(n_epochs):
        loss_train, acc_train = run_train(model, optimizer, scheduler)
        loss_test, acc_test = run_test(model)
        print(f"epoch {epoch}: train loss {loss_train:.4f} acc {acc_train:.4f}, test loss {loss_test:.4f} acc {acc_test:.4f}")

In [68]:
run_all(model, optimizer, scheduler, 10)

RuntimeError: ignored